# Introduction to SCoPe and its periodic variables: ZTF Summer School

### Keaton Bell <keatonb@uw.edu>, Michael Coughlin <cough052@umn.edu>


# Introduction to frequency analysis of variable light curves in the ZTF variable source catalog

This tutorial will provide a brief introduction to frequency analysis of variable star light curves, like those being obtained from ZTF.  We will demonstrate how to obtain a multi-sinusoid fit to time series data, how to deal with potential challenges from gaps in the data, and we'll conclude with a discussion of signal significance.  We will use the packages [astroquery](https://astroquery.readthedocs.io/en/latest/), [ztfquery](https://github.com/MickaelRigault/ztfquery), [lightkurve](http://docs.lightkurve.org/), and [Pyriod](https://github.com/keatonb/Pyriod/), so you'll need those installed, and you'll also need an IRSA account to download ZTF data.

After you install Pyriod, make sure to enable the following Jupyter extensions in the terminal (before you start your Jupyter Notebook server!):
```
jupyter nbextension enable --py --sys-prefix qgrid
jupyter nbextension enable --py --sys-prefix widgetsnbextension
jupyter nbextension enable --py --sys-prefix ipympl
```

Pyriod works in Jupyter Notebooks, but not in Jupyter Lab. The widgets require the `%matplotlib widget` magic command to be called.

First, since we want to analyze variable light curves, we'll use SCoPe published in [van Roestel et al. (2021, AJ, 161 267)](https://iopscience.iop.org/article/10.3847/1538-3881/abe853/meta). 

In [22]:
%matplotlib widget

In [3]:
import pandas as pd
DATA_PATH = './data/dataset.csv'
df_raw_data = pd.read_csv(DATA_PATH)

In [4]:
LABELS_COL = [# 10 Cols for the phenomenological classifiers 
              'variable', 'periodic', 'long timescale', 'irregular', 'eclipsing', 'EA', 'EB', 'EW', 'flaring', 'bogus',
              # new
              'non-variable', 'dipping', 'blend', 'bright star', 'ccd artifact', 'galaxy', 'sinusoidal', 'sawtooth', 
              'elliptical',
    
    
              # 13 Cols for the ontological classifiers
              'pulsator', 'Delta Scu', 'Cepheid', 'RR Lyrae', 'LPV', 'Mira', 'SRV', 'binary star', 'W Uma', 'Beta Lyr',
              'RS CVn', 'AGN', 'YSO',
              # new
              'F', 'O', 'Cepheid type-II', 'detached eclipsing MS-MS', 'compact binary', 'eclipsing WD+dM (NN Ser)',
              'eclipsing sdB+dM (HW Vir)', 'RR Lyrae Blazhko', 'RR Lyrae ab', 'RR Lyrae c', 'RR Lyrae d', 'BL Her', 
              'RV Tau', 'W Virginis',  
    
              # unclear:
              'double period', 'half period', 'multi periodic', 'nice', 'niice', 'wrong period',
             ]

FEATS_COL = [# 38 Cols for the phenomenological classifiers 
             'period', 'significance', 'n', 'median', 'wmean', 'wstd', 'chi2red', 'roms', 'norm_peak_to_peak_amp',
             'norm_excess_var', 'median_abs_dev', 'iqr', 'f60', 'f70', 'f80' , 'f90', 'skew', 'smallkurt', 
             'inv_vonneumannratio', 'welch_i', 'stetson_j', 'stetson_k', 'ad', 'sw', 'f1_power', 'f1_bic','f1_amp', 
             'f1_phi0', 'f1_relamp1', 'f1_relphi1', 'f1_relamp2', 'f1_relphi2', 'f1_relamp3', 'f1_relphi3', 'f1_relamp4', 
             'f1_relphi5', 'n_ztf_alerts', 'mean_ztf_alert_braai',  
             
             # 30 Cols for the ontological classifiers
             'AllWISE__w1mpro', 'AllWISE__w1sigmpro', 'AllWISE__w2mpro', 'AllWISE__w2sigmpro', 'AllWISE__w3mpro',
             'AllWISE__w3sigmpro', 'AllWISE__w4mpro','AllWISE__w4sigmpro', 'AllWISE__ph_qual',
             'Gaia_DR2__phot_g_mean_mag', 'Gaia_DR2__phot_bp_mean_mag', 'Gaia_DR2__phot_rp_mean_mag', 'Gaia_DR2__parallax',
             'Gaia_DR2__parallax_error', 'Gaia_DR2__pmra', 'Gaia_DR2__pmra_error', 'Gaia_DR2__pmdec', 'Gaia_DR2__pmdec_error',
             'Gaia_DR2__astrometric_excess_noise', 'Gaia_DR2__phot_bp_rp_excess_factor',
             'PS1_DR1__gMeanPSFMag', 'PS1_DR1__gMeanPSFMagErr', 'PS1_DR1__rMeanPSFMag', 'PS1_DR1__rMeanPSFMagErr', 
             'PS1_DR1__iMeanPSFMag', 'PS1_DR1__iMeanPSFMagErr', 'PS1_DR1__zMeanPSFMag', 'PS1_DR1__zMeanPSFMagErr', 
             'PS1_DR1__yMeanPSFMag', 'PS1_DR1__yMeanPSFMagErr', 'PS1_DR1__qualityFlag']


df_feats = df_raw_data[FEATS_COL]
df_labels = df_raw_data[LABELS_COL]

In [5]:
ra, dec = df_raw_data['ra'], df_raw_data['dec']

## Find number of objects of each type

NB: an object label can be 0, 0.25, 0.5, 0.75, or 1 depending on the labelers'certainty

In [6]:
import numpy as np

for column in df_labels:
    print(column, np.sum(df_labels[column]))

variable 44054.710999999996
periodic 36988.5
long timescale 523.75
irregular 948.8749999999999
eclipsing 25089.625
EA 459.0
EB 524.5
EW 21581.625
flaring 450.75
bogus 784.625
non-variable 25592.664
dipping 43.5
blend 227.25
bright star 178.0
ccd artifact 17.5
galaxy 163.75
sinusoidal 628.0
sawtooth 577.0
elliptical 524.0
pulsator 10983.875
Delta Scu 3459.0
Cepheid 523.75
RR Lyrae 6573.0
LPV 353.75
Mira 90.75
SRV 242.75
binary star 25030.0
W Uma 21514.25
Beta Lyr 457.5
RS CVn 665.0
AGN 344.5
YSO 425.9583333333333
F 14.0
O 0.0
Cepheid type-II 72.5
detached eclipsing MS-MS 2077.0
compact binary 7.5
eclipsing WD+dM (NN Ser) 6.5
eclipsing sdB+dM (HW Vir) 0.5
RR Lyrae Blazhko 54.5
RR Lyrae ab 1870.0
RR Lyrae c 2567.0
RR Lyrae d 205.0
BL Her 14.0
RV Tau 0.5
W Virginis 12.75
double period 435.5
half period 166.0
multi periodic 125.25
nice 21.0
niice 5.0
wrong period 742.0


Choose an object from the table, and let's download and analyze the data.  Try a random one or browse the table for something that interests you.

In [ ]:
import numpy as np
ind = np.random.choice(np.arange(len(df_raw_data))) #Random row number
#ind = 368056 #For tutorial video
row = df_raw_data.iloc[ind]
row

You'll need an IRSA account to access the data.  Download and display the available data as follows.

In [ ]:
from ztfquery import lightcurve
lcq = lightcurve.LCQuery()
result = lcq.query_position(row["ra"], row["dec"], 3)
data = result.data
result.show_lc() #Display a light curve

A Jupyter widget could not be displayed because the widget state could not be found. This could happen if the kernel storing the widget is no longer available, or if the widget state was not saved in the notebook. You may be able to create the widget by running the appropriate cells.

We'll analyze data taken in a single filter in this tutorial, so pick one that has plenty of data in the plot above. The filter codes are `zg` (green above), `zr` (red), and `zi` (orange).  We will also reject any points that have corresponding catalog flags indicating potentially poor quality data.

In [ ]:
filtercode = 'zr' #just this filter
catflags = 0 #reliable data only (hopefully)
datatable = data[(data.filtercode == filtercode) & (data.catflags == catflags)]

#Look at the contents of the resulting table
datatable

So there's all the time series data for our chosen filter.  We need to put this in a form that Pyriod will understand, i.e., the `lightkurve` package's `LightCurve` object.  But before that, we'll want to convert from magnitude to flux units for our analysis (analyzing magnitudes is okay, this is just my preference).  We'll normalize the light curves (to be centered around 1.0), so we don't need to worry about the flux zero point, so I just use some arbitrary reference magnitude.

In [ ]:
time = datatable['mjd']
flux = 10**(-0.4*(datatable['mag']-21))
flux_err = flux * datatable['magerr'] / 1.1816

import lightkurve as lk
lc = lk.LightCurve(time = time, flux = flux, flux_err = flux_err)
lc = lc.normalize() #normalize so centered on 1
lc.scatter() #plot the light curve data

A Jupyter widget could not be displayed because the widget state could not be found. This could happen if the kernel storing the widget is no longer available, or if the widget state was not saved in the notebook. You may be able to create the widget by running the appropriate cells.

Okay, our light curve is now in units of relative flux.  We're finally ready to pass this on to Pyriod's interactive frequency analysis tools.

In [ ]:
from Pyriod import Pyriod
pyriod = Pyriod(lc, freq_unit='day', maxfreq = 100)

Pyriod has four interactive "cells" that let you interact with the data and obtain a multi-sinusoid fit.  They are the TimeSeries, Periodogram, Signals, and Log cells, each displayed in the next four cells. Click the "info" drop-down for more information about how to interact with each cell. A demonstration of how to use these tools to obtain a reliable fit, watch the accompanying video.  If you want to develop a better understanding of the Lomb-Scargle periodogram, check out [Vanderplas (2018, ApJS, 236, 16)](https://ui.adsabs.harvard.edu/abs/2018ApJS..236...16V).

In [ ]:
pyriod.TimeSeries()

In [ ]:
pyriod.Periodogram()

In [ ]:
pyriod.Signals()

In [ ]:
pyriod.Log()

In addition to those tools, we need some way of assessing the significance of peaks in the periodgram so we can decide whether to concorporate them into our analysis.  We'll use a technique called bootstrapping, where we make the assumption that the residuals contain only uncorrelated noise to see how high of a periodogram peak such noise could reasonably produce.  We treat the data as noise by drawing randomly from the light curve new "noisy" light curves that are sampled at the same times as the original, destroying any coherent signals.  If we find higher peaks in the periodogram of our data, we can trust to some level of confidence that it corresponds to a real underlying signal.  That doesn't mean the highest peak is at the frequency of variability!  A different test that injects signals into the bootstrapped noise model and checks how often the highest peak corresponds to the injected frequency can test for this criterion.  Also beware of a heightened noise level at low frequencies, which can be due to long-timescale noise in the data. It's often a better idea to determine a significance threshold as a multiple of the local mean periodogram values. Five times the local mean level in the periodogram will probably yield reliable pleaks.  This process can take hours, but it's worth it to recalculate a new significance threshold when there are no longer peaks above the previously calculated value.

In [ ]:
from tqdm import tqdm #status bar
fap = 0.01 #False alarm probability
nruns = 100
highestpeaks = np.zeros(nruns)
noisedistribution = 1 + pyriod.lc.resid[np.where(pyriod.lc["mask"])].value

for i in tqdm(range(nruns)): #~3 hours
    bootstrappedlc = lc.copy()
    bootstrappedlc.flux = np.random.choice(noisedistribution,len(lc)) #with replacement
    bootstrappedlc = bootstrappedlc.normalize() #re-normalize
    per = bootstrappedlc.to_periodogram(normalization='amplitude',freq_unit=pyriod.freq_unit,
                                        frequency=pyriod.freqs)*pyriod.amp_conversion
    highestpeaks[i] = per.max_power.value
    
#print the peak height corresponding to the desired False Alarm Probability
np.percentile(highestpeaks,100*(1-fap))